In [1]:
import os
import pandas as pd
from convert_to_csv import convert_to_csv

In [2]:
convert_to_csv(prefix="../")
df = pd.read_csv("../midterm-2018/data.csv")
df.head()

            probe_timestamp     user_id    screen_name               name  \
0  Tue Nov 06 20:35:08 2018  4107317134  danitheduck21           Dani🏳️‍🌈   
1  Tue Nov 06 17:57:51 2018  4858296837   ncaraballoPR  Natalie Caraballo   
2  Tue Nov 06 20:35:23 2018   232631847     drmendezmd             Wilson   
3  Tue Nov 06 19:23:19 2018    16700555    ScottNevins       Scott Nevins   
4  Tue Nov 06 20:35:24 2018   334443152       lild1206                  D   

                                         description  \
0  Dani 💜 She/Her 💜 Randomness all over. Expect l...   
1  Things I don’t get tired of: Politics, Amy Win...   
2  Latin american100%! Let fight for our country,...   
3  TV Personality & Host | Political/News Contrib...   
4                                               None   

            user_created_at                         url lang protected  \
0  Tue Nov 03 21:16:13 2015                        None   en     False   
1  Thu Jan 28 20:03:51 2016                        N

C:\Users\kalmu\AppData\Local\Temp/ipykernel_19956/1013483409.py:2: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../midterm-2018/data.csv")


,Unnamed: 0,probe_timestamp,user_id,screen_name,name,description,user_created_at,url,lang,protected,...,geo_enabled,profile_use_background_image,default_profile,followers_count,friends_count,listed_count,favourites_count,statuses_count,tid,label
0,0,Tue Nov 06 20:35:08 2018,4107317134,danitheduck21,Dani🏳️‍🌈,Dani 💜 She/Her 💜 Randomness all over. Expect l...,Tue Nov 03 21:16:13 2015,NaN,en,False,...,False,False,False,481,870,26,6542,67025,1059907055421509632,human
1,1,Tue Nov 06 17:57:51 2018,4858296837,ncaraballoPR,Natalie Caraballo,"Things I don’t get tired of: Politics, Amy Win...",Thu Jan 28 20:03:51 2016,NaN,en,False,...,False,False,False,202,712,5,1515,158,1059867472810180609,human
2,2,Tue Nov 06 20:35:23 2018,232631847,drmendezmd,Wilson,"Latin american100%! Let fight for our country,...",Fri Dec 31 18:55:05 2010,NaN,en,False,...,True,True,True,278,342,4,4780,4029,1059907117094711296,human
3,3,Tue Nov 06 19:23:19 2018,16700555,ScottNevins,Scott Nevins,TV Personality & Host | Political/News Contrib...,Sat Oct 11 21:39:34 2008,http://www.ScottNevins.com,en,False,...,False,True,False,29546,384,402,143163,53427,1059888980957650944,human
4,4,Tue Nov 06 20:35:24 2018,334443152,lild1206,D,NaN,Wed Jul 13 03:13:52 2011,NaN,en,False,...,True,True,True,95,668,1,1178,1315,1059907122408898562,human
